In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import umap

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from mpl_toolkits.mplot3d import Axes3D


import sys
# sys.path.append("/Users/Work/Developer/interpretDL/interprettensor")
root_logdir = "./tf_logs"
datadir = "data/"
figures_dir = "data/figures/"

# To plot pretty figures
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

np.random.seed(seed=42) 
tf.__version__

'1.13.1'

In [10]:
# Helper Functions

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

######### Taken from sklearn #######
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=[8,8])
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


def get1hot(y_train,y_test):
    from sklearn.preprocessing import OneHotEncoder

    enc = OneHotEncoder(categories="auto", sparse=False)
    y_train_1hot = enc.fit_transform([[label] for label in y_train]) # Since the function expects an array of "features" per sample
    y_test_1hot = enc.fit_transform([[label] for label in y_test])

    return y_train_1hot, y_test_1hot

def get_split(features, labels):
    features = np.array(features)
    labels = np.array(labels)
    # The train set will have equal amounts of each target class
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(features, labels):
        X_train = features[train_index]
        y_train = labels[train_index]
        X_test = features[test_index]
        y_test = labels[test_index]
        
        yield X_train, y_train, X_test, y_test

def plot_history(history):
    plt.close("History")
    fig, axs = plt.subplots(1, 2, figsize=(12,6),num="History")

    # Plot training & validation accuracy values
    axs[0].grid(True)
    axs[0].plot(history.history['binary_accuracy'])
    axs[0].plot(history.history['val_binary_accuracy'])
    axs[0].set(title='Model accuracy', ylabel='Accuracy', xlabel='Epoch')
    axs[0].legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    axs[1].grid(True)
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set(title='Model loss',ylabel='Loss', xlabel='Epoch')
    axs[1].legend(['Train', 'Test'], loc='upper left')

    plt.show()


def remove_label(features, labels, label="MCI"):
    labels = pd.Series(fused_labels)
    non_samples = labels != label

    stripped_features = features[non_samples]
    stripped_labels = labels[non_samples]

    return stripped_features, stripped_labels

In [11]:
from sklearn.datasets import make_circles
from sklearn.datasets import make_moons
from sklearn.datasets.samples_generator import make_blobs
from matplotlib import pyplot
from pandas import DataFrame

'''
Returns orginal samples, labels and modded_samples,modded_labels
'''
def modded_iris():

    from sklearn import datasets

    iris = datasets.load_iris()

    features = pd.DataFrame(iris["data"])
    target = pd.Series(iris["target"])
    flower_names = iris["target_names"]
    feature_names = iris["feature_names"]
    print(features.info())

    ### Get the first 2 flower samples

    setosa = target == 0
    versicolor = target == 1
    samples = features[setosa | versicolor]
    labels = target[setosa | versicolor]
    class_size = sum(versicolor)

    versicolor_samples = features[versicolor]
    versicolor_labels = target[versicolor]
    setosa_samples = features[setosa]

    ### Splitting *versicolor* into two sub classes

    versicolor_samples.describe()

    ## Constructing different noise sources
    gauss_noise = np.random.normal(loc=1,scale=0.25, size=[class_size//2,2])
    gauss_noise[gauss_noise < 0] = 0
    unif_noise = np.random.uniform(low=0,high=1)
    constant = 1


    split_size = class_size//2

    # Positive to first two features

    B1 = versicolor_samples.iloc[:split_size,:2] + gauss_noise
    B1 = np.concatenate((B1, versicolor_samples.iloc[:split_size,2:]), axis=1)
    B1_labels = versicolor_labels.iloc[:split_size]

    # Negative to last two features
    # gauss_noise = np.random.normal(loc=0.1,scale=0.1, size=[class_size//2,2])
    # gauss_noise[gauss_noise < 0] = 0
    # unif_noise = np.random.uniform(low=0,high=1)

    # B2 = versicolor_samples.iloc[split_size:,2:] + gauss_noise
    # B2 = np.concatenate((versicolor_samples.iloc[split_size:,2:],B2), axis=1)

    B2 = versicolor_samples.iloc[split_size:,:2] - gauss_noise
    B2 = np.concatenate((B2,versicolor_samples.iloc[split_size:,2:]), axis=1)
    B2_labels = versicolor_labels.iloc[split_size:] + 1

    # Combining the two fake "subclasses"
    noisy_samples = np.concatenate((B1, B2), axis=0)


    modded_samples = np.concatenate((setosa_samples,noisy_samples))
    modded_labels = labels.copy()
    modded_labels[class_size + split_size:] += 1

    return samples,labels,modded_samples, modded_labels


'''
Returns 8 gaussian blobs surrounding one center blob

       labels: Only center vs other labels (0,1) 
modded_labels: The labels for all 9 classes
'''
def simulate_blobs(class_size = 200):
    centers = [2*(x,y) for x in range(-1,2) for y in range(-1,2)]
    n_samples = [class_size//(len(centers)-1)]*len(centers)
    n_samples[len(centers)//2] = class_size

    X, y = make_blobs(n_samples=n_samples, centers=centers, n_features=2, cluster_std=0.1, shuffle=False, random_state=42)

    plt.close(1)
    df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
    fig, ax = plt.subplots()
    colors = {0:'red', 1:'blue'}
    df.plot(ax=ax,kind="scatter", x='x', y='y',c="label", cmap= "Paired")
    # plt.colorbar()
    plt.show()
    
    original_labels = df["label"].copy()
    modded_samples = df[["x","y"]].copy()
    labels = df["label"].copy()
    labels[labels != 4] = 0
    labels[labels == 4] = 1
    return df, modded_samples,labels, original_labels

## Using make_moons to generate sample data

In [12]:
def get_split_index(features, labels):
    features = np.array(features)
    # The train set will have equal amounts of each target class
    # Performing single split
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    return [[train_index, test_index] for train_index,test_index in split.split(features, labels)]

def split_valid(features, original_labels, training_labels):
    train_index, validation_index = get_split_index(features, original_labels)[0]
    
    X_valid, y_valid, y_valid_original = features.iloc[validation_index],  training_labels.iloc[validation_index], original_labels.iloc[validation_index]
    X_train, y_train, y_original = features.iloc[train_index], training_labels.iloc[train_index], original_labels.iloc[train_index]
     
    return X_train, y_train, y_original, X_valid, y_valid, y_valid_original

def get_train_test_val(features, original_labels, training_labels):
    
    X, y, y_original, X_valid, y_valid, y_valid_original = split_valid(features,original_labels, training_labels)
   
    train_index, test_index = get_split_index(X, y_original)[0]
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    return X_train, y_train, X_test, y_test, y_original, X_valid, y_valid, y_valid_original

### Train a DNN on the modified dataset

In [13]:
# Get split returns a generator
# List comprehension is one way to evaluate a generator

original_data, modded_samples, training_labels, original_labels = simulate_blobs(class_size=800)

# Separating a hold out set that will be used for validation later
X_train, y_train, X_test, y_test, y_original, X_valid, y_valid, y_valid_original = get_train_test_val(modded_samples, original_labels, training_labels)


print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)


hot_encoder = OneHotEncoder(categories="auto", sparse=False)
hot_encoder.fit(training_labels.values.reshape(-1,1)) # Since the function expects an array of "features" per sample
print("Categories:", hot_encoder.categories_)

FigureCanvasNbAgg()

Train Size: (1024, 2)
Test Size: (256,)
Categories: [array([0, 1])]


In [14]:
NUM_FEATURES = X_train.shape[1]
NUM_LABELS = len(hot_encoder.categories_[0])

In [15]:
def build_dnn(num_features, num_labels=3):

#     reset_graph()
    
    keras.backend.clear_session()

    nn = keras.models.Sequential()
    Dense = keras.layers.Dense
    
    # Using He initialization
    he_init = tf.keras.initializers.he_uniform()
    
    nn.add(Dense(units = 12, activation="elu", input_dim=num_features,
                kernel_initializer=he_init))
    nn.add(Dense(units = 12, activation="elu",
                kernel_initializer=he_init))
    nn.add(Dense(units=1, activation= "sigmoid",
                kernel_initializer=he_init))

#     BCE = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    nn.compile(loss="binary_crossentropy",
                  optimizer='sgd',
                  metrics=['binary_accuracy'])
    
    return nn

def train_model(model, X, y, X_test=[], y_test=[], epochs=30, batch_size=20, verbose=1, plot=True):
    
    ZScaler = StandardScaler().fit(X)
    
    X_train = ZScaler.transform(X)
    X_test = ZScaler.transform(X_test)
    
    y_train = np.array(y)
    y_test = np.array(y_test)
    
#     lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay)
    callback_list = []
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size,
                        validation_data=(X_test, y_test), callbacks=callback_list, verbose=verbose)
    
#     if plot: plot_history(history)
    
    return history, ZScaler


In [16]:
nn = build_dnn(NUM_FEATURES)
%time history, Zscaler = train_model(nn, X_train, y_train, X_test, y_test, epochs=50, batch_size=20)

Instructions for updating:
Use tf.cast instead.
Train on 1024 samples, validate on 256 samples
Epoch 1/50
1024/1024 [==============================] - 0s 230us/step - loss: 0.4562 - binary_accuracy: 0.8711 - val_loss: 0.4030 - val_binary_accuracy: 0.9609
Epoch 2/50
1024/1024 [==============================] - 0s 42us/step - loss: 0.3725 - binary_accuracy: 0.9766 - val_loss: 0.3359 - val_binary_accuracy: 0.9922
Epoch 3/50
1024/1024 [==============================] - 0s 43us/step - loss: 0.3128 - binary_accuracy: 0.9932 - val_loss: 0.2827 - val_binary_accuracy: 0.9961
Epoch 4/50
1024/1024 [==============================] - 0s 40us/step - loss: 0.2647 - binary_accuracy: 0.9980 - val_loss: 0.2394 - val_binary_accuracy: 1.0000
Epoch 5/50
1024/1024 [==============================] - 0s 40us/step - loss: 0.2254 - binary_accuracy: 0.9980 - val_loss: 0.2038 - val_binary_accuracy: 1.0000
Epoch 6/50
1024/1024 [==============================] - 0s 41us/step - loss: 0.1930 - binary_accuracy: 0.9980

In [17]:
# Plotting results from history
plot_history(history)

FigureCanvasNbAgg()

In [18]:
preds = [round(x[0]) for x in nn.predict(Zscaler.transform(X_test))]
_labels = [np.float(x) for x in y_test]
preds[:5],_labels[:5]

([1.0, 0.0, 0.0, 1.0, 1.0], [1.0, 0.0, 0.0, 1.0, 1.0])

# DONT FORGET TO RENORMALIZE WHEN EVALUATING SAMPLES

In [19]:
# from sklearn.model_selection import StratifiedKFold as KFold
# # keras.backend.clear_session()

# def getKF(X,y, n_splits=10):
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=42 ) #Default = 10

#     for train_index, test_index in kf.split(X,y):
#         X_train = X[train_index]
#         y_train = y.iloc[train_index]
#         X_test = X[test_index]
#         y_test = y.iloc[test_index]
        
#         yield X_train, y_train, X_test, y_test, test_index

# histories = []
# testing_indxs =[]
# predictions = []
# true_labels = []
# zoo = []
# for X_train, y_train, X_test, y_test, test_index in getKF(modded_samples, labels):
#     print(test_index)
#     dnn = build_dnn(NUM_FEATURES, NUM_LABELS)
#     history, ZScaler = train_model(dnn,X_train, y_train, X_test, y_test, verbose=0, plot=False, epochs=20, batch_size=10)
    
#     # Updating all information arrays
#     histories.append(history)
#     testing_indxs.append(test_index)
#     zoo.append(dnn)
    
#     y_pred_probs = dnn.predict(ZScaler.transform(X_test))
#     y_pred = np.argmax(y_pred_probs, axis=1)
#     y_true = np.argmax(hot_encoder.transform(y_test.values.reshape(-1,1)), axis=1)
    
#     predictions.extend(y_pred)
#     true_labels.extend(y_true)
    
#     print("Scores on test set: loss={:0.3f} accuracy={:.4f}".format(history.history["binary_accuracy"][-1], history.history["val_binary_accuracy"][-1]))

In [20]:
# # Num is the figure number and clear tells it to clear the figure if it already exists
# plt.close()
# fig, axs = plt.subplots(num="KF Eval",
#                         nrows=len(histories)//2, ncols=2,
#                         figsize=(10,10), sharex=True, sharey=True)
# axs=axs.flatten()
# dfs = []

# for i,history in enumerate(histories):
#     df = pd.DataFrame(history.history)
#     dfs.append(df)
# #     axs[i].grid(True)
#     df[["binary_accuracy","val_binary_accuracy"]].plot(ax=axs[i], grid=True)

In [21]:
# sess = tf.Session()
# idx = -1
# best_dnn = zoo[idx]

# predictions = best_dnn.predict(Zscaler.transform(modded_samples))
# _labels = [np.float(x) for x in labels]
# sess.run(keras.metrics.binary_accuracy(_labels,predictions.flatten()))

# best_dnn.evaluate(Zscaler.transform(modded_samples),labels.values)
# # NOT bestdnn.evaluate(modded_samples,labels.values)


## Performing SVM on Modded Samples

In [22]:
from sklearn.svm import LinearSVC

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("SVM", LinearSVC(C=1, loss="hinge", max_iter=1000 ))
])

%time svm_clf.fit(X_train, y_train)
print("Linear SVM Test Accuracy: {:0.3f}".format(svm_clf.score(X_test, y_test)))

CPU times: user 5.21 ms, sys: 1.67 ms, total: 6.88 ms
Wall time: 5.37 ms
Linear SVM Test Accuracy: 0.688


## Performing LRP

In [23]:
model = nn
scaled_samples = Zscaler.transform(X_valid)
_labels = y_valid
# mod_labels = modded_labels[test_index]

predictions = model.predict(scaled_samples)
preds = np.array([np.round(x[0]) for x in predictions])
true_labels = [np.float(x) for x in _labels]

correct = preds == true_labels
# versicolor = true_labels == 1

print("Validation Accuracy")
loss_and_metrics = model.evaluate(scaled_samples, y_valid)
print("Scores on validation set: loss={:0.3f} accuracy={:.4f}".format(*loss_and_metrics))

Validation Accuracy
320/320 [==============================] - 0s 18us/step
Scores on validation set: loss=0.011 accuracy=1.0000


In [24]:
_labels[correct].value_counts()

1    160
0    160
Name: label, dtype: int64

In [25]:
import innvestigate
import innvestigate.utils as iutils

def perform_analysis(model, analyzer, data, labels=[]):
    analysis = analyzer.analyze(data)
    prediction = model.predict(data)
    
    df_anal = pd.DataFrame(analysis)
    
    return df_anal


# Stripping the softmax activation from the model
# model_wo_sm = iutils.keras.graph.model_wo_softmax(model)

# Creating an analyzer
lrp_E = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model=model, epsilon=1e-3)
lrp_Z = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPZPlus(model=model)
lrp_AB   = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha2Beta1(model=model)

# Getting all the samples that can be correctly predicted
test_idx = correct
all_samples = scaled_samples[test_idx]
all_labels = y_valid_original[test_idx]


# perform_analysis(nn,gradient_analyzer,flowers,types)
all_lrp_AB = perform_analysis(model,lrp_AB, all_samples)
all_lrp_E = perform_analysis(model,lrp_E, all_samples)
all_lrp_Z = perform_analysis(model,lrp_Z, all_samples)


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [29]:
plt.close("LRP Analysis")
fig, axs = plt.subplots(2,2, figsize=(16,10), num="LRP Analysis")
_labels = all_labels
# df = DataFrame(modded_samples)

original_data.plot(ax=axs[0][0],kind="scatter", x='x', y='y',c="label", cmap= "Paired", title="Original Distribution")
all_lrp_E.plot(ax=axs[0][1],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP E")

all_lrp_AB.plot(ax=axs[1][0],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP AB")
all_lrp_Z.plot(ax=axs[1][1],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP Z")

plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

In [38]:
all_lrp_E.plot(figsize=(16,10),kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=30, alpha=0.85, title="LRP E")
plt.savefig(figures_dir+"8su_lrpE.png")

FigureCanvasNbAgg()

In [39]:
# plt.savefig(figures_dir+"8su_lrpE_zoomed.png")